In [1]:
import nltk;
import string
import sys
import pandas as pd
import networkx as nx
import sklearn.decomposition as dec
import sklearn.manifold as manif
import matplotlib.pyplot as plt
import mpld3
import scipy.sparse.linalg as linalg
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import time
import json
import glob
import re
import os
import itertools 
import pickle

In [2]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag, wordnet
from summa import keywords
from summa.summarizer import summarize
from nltk.tokenize import MWETokenizer
from pybabelfy.babelfy import *
from nltk.corpus import stopwords
from pybabelnet.babelnet import BabelNet
from unidecode import unidecode    
from collections import OrderedDict
#from nltk.stem import PorterStemmer
from porterStemmer import PorterStemmer
from math import log


In [3]:
def openFile(fileName):
    file = open(fileName,"r") 
    text = file.read() 
    file.close()
    return text

In [4]:
def parseText(texto):
    text = ""
    punctuation = string.punctuation
    punctuation = punctuation.replace('!','')
    punctuation = punctuation.replace('.','')
    punctuation = punctuation.replace('?','')
    punctuation = punctuation.replace('-','')
    translator = str.maketrans('', '', punctuation)
    
    for l in texto.splitlines():
        printable = "".join(filter(str.isprintable,l))
        text = text + '\n' + printable.translate(translator)
    text = unidecode(text)
    text = text.replace('%','')
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'  ', ' ', text)
    splitted_text = [s.strip() for s in text.splitlines()]
    new_splitted_text = []
    punctuation = ['.','?','!']
    new_splitted_text.append("")
    for line in text.splitlines():
        if new_splitted_text[-1][-1:] not in punctuation or len(new_splitted_text[-1] + line) <= 3000: 
            new_splitted_text[-1] = new_splitted_text[-1] + ' ' + line
        else:
            new_splitted_text.append(line)
    translator = str.maketrans('', '', "?!.")
    for l in new_splitted_text:
        l = l.translate(translator)
    return new_splitted_text

In [5]:
def frag(semantic_annotation, input_text):
    start = semantic_annotation.char_fragment_start()
    end = semantic_annotation.char_fragment_end()
    return input_text[start:end+1]

In [6]:
def babelfy(lang, KEY, splitted_text):
    babelapi = Babelfy()
    bn = BabelNet(key)
    paragraphs_annotations = []
    paragraphs_text = []
    paragraphs_code = []
    
    count = 0
    for paragraph in splitted_text: #annotate each paragraph
        words_annotations = []
        words_text = []
        words_code = []
        semantic_annotations = babelapi.disambiguate(paragraph,lang,key,match="EXACT_MATCHING",cands="TOP",mcs="ON",anntype="ALL")
        
        for semantic_annotation in semantic_annotations: #exclude unused annotations (multiword expressions)
            # primeira palavra ou  palavra atual comeca apos anteior
            if len(words_annotations) == 0 or words_annotations[-1].char_fragment_end() < semantic_annotation.char_fragment_start():
                words_annotations.append(semantic_annotation)
                #words_text.append(bn.getSynsets(semantic_annotation.babel_synset_id())[0].senses[0].properties.fullLemma.lower())
                words_text.append(frag(semantic_annotation,paragraph))
                words_code.append(semantic_annotation.babel_synset_id())

            # palavra atual comeca junto da palavra anterior
            elif words_annotations[-1].char_fragment_start() == semantic_annotation.char_fragment_start():
                del words_annotations[-1]
                words_annotations.append(semantic_annotation)
                del words_text[-1]
                #words_text.append(semantic_annotation.babel_synset_id())[0].senses[0].properties.fullLemma.lower())
                words_text.append(frag(semantic_annotation,paragraph))
                del words_code[-1]
                words_code.append(semantic_annotation.babel_synset_id())


        paragraphs_annotations.append(words_annotations)
        paragraphs_text.append(words_text)
        paragraphs_code.append(words_code)
        count = count + 1
        print(count)
        
    return paragraphs_annotations, paragraphs_text, paragraphs_code    

In [7]:
def saveText(file, paragraphs_code):
    file = open(file,"w+") 
    for p in paragraphs_code:
        for w in p:
            file.write(w + "#")
        file.write("\n")
    file.close()

 ## dicionario[palavra] = codigo
 ## dicionarioCode[codigo] = palavra
 ## peso[codigo] = peso

In [8]:
def createDicts(paragraphs_text, paragraphs_code):
    dicionario={}
    peso={}
    dicionarioCode={}
    for paragraph, codes in zip(paragraphs_text, paragraphs_code):
        for word, code in zip(paragraph, codes):
            if code not in peso or word not in dicionario:
                dicionario[word] = code
                peso[code] = 1
                if code not in dicionarioCode:
                    dicionarioCode[code] = word
            else:
                peso[code] = peso[code] + 1   
    return dicionario, dicionarioCode, peso

In [9]:
float(1 - log(3,4))

0.20751874963942185

In [10]:
g = nx.DiGraph()
g.add_edge(1,2, weight=1)
g.add_edge(1,3, weight=0.5+0.20751874963942185)
g.add_edge(2,3, weight=1.5)
g.add_edge(2,4, weight=0.20751874963942185)
g.add_edge(3,4, weight=1.5)
g.add_edge(3,5, weight=0.5+0.20751874963942185)
g.add_edge(3,6, weight=0.20751874963942185)
g.add_edge(4,5, weight=1)
g.add_edge(4,6, weight=0.5)
g.add_edge(4,7, weight=0.20751874963942185)
g.add_edge(5,6, weight=1)
g.add_edge(5,7, weight=0.5)
g.add_edge(5,8, weight=0.20751874963942185)
g.add_edge(6,7, weight=1)
g.add_edge(6,8, weight=0.5)
g.add_edge(6,9, weight=0.20751874963942185)
g.add_edge(7,8, weight=1)
g.add_edge(7,9, weight=0.5)
g.add_edge(8,9, weight=1)

In [11]:
pr = nx.degree_centrality(g)

In [12]:
pr

{1: 0.25,
 2: 0.375,
 3: 0.625,
 4: 0.625,
 5: 0.625,
 6: 0.75,
 7: 0.625,
 8: 0.5,
 9: 0.375}

In [13]:
def createGraph(peso, paragraphs_code, dist):
    g = nx.DiGraph()
    g2 = nx.DiGraph()
    g3 = nx.DiGraph()
    for code, weight in peso.items():
        g.add_node(code, peso=weight)
        g2.add_node(code, peso=weight)
        g3.add_node(code, peso=weight)
    for line in paragraphs_code:
        i = 0
        for word in line:
            i = i + 1
            j = 0
            for word2 in line:
                j = j + 1
                if j - i < dist and j - i > 0:
                    if g.has_edge(word, word2):
                        #g[word][word2]['weight'] += 1/(j-i)
                        g[word][word2]['weight'] += 1 - log(j-i,dist)
                    else:
                        if word != word2:
                            #g.add_edge(word, word2, weight=float(1/(j-i)))
                            g.add_edge(word, word2, weight=float(1 - log(j-i,dist)))
                            #g.add_edge(word, word2, weight=float(log(dist/j-i,10)))
                if j - i == 1:
                    if g2.has_edge(word, word2):
                        g2[word][word2]['weight'] += 1
                    else:
                        if word != word2:
                            g2.add_edge(word, word2, weight=1)
                if j - i == 2:
                    if g3.has_edge(word, word2):
                        g3[word][word2]['weight'] += 1
                    else:
                        if word != word2:
                            g3.add_edge(word, word2, weight=1)
    ## Normalize edge weights
    #arcs = 0
    #for n1, n2, w in list(g.edges(data='weight')):
    #    arcs = arcs + w
    #for a in list(g.edges(data='weight')):
    #    g[n1][n2]['weight'] = g[n1][n2]['weight'] * 100 / arcs 

    return g, g2, g3
    

In [14]:
def nodeRank(g,dicionarioCode):
    #pr = nx.pagerank(g, weight='weight')
    #pr = nx.eigenvector.eigenvector_centrality_numpy(g, weight='weight')
    #pr = nx.closeness_centrality(g, distance='weight')
    pr = nx.degree_centrality(g)
    desc = sorted(pr.keys(), key=pr.get,reverse=False)
    pageRank = {}
    for p in pr:
        pageRank[p] = pr[str(p)] 
    pageRank = sorted(pageRank.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
    return pageRank

    #desc = sorted(pr.keys(), key=pr.get,reverse=False)
    #pageRank = {}
    #for p in pr:
    #    pageRank[dicionarioCode[p]] = pr[str(p)] 
    #pageRank = sorted(pageRank.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
    #return pageRank

In [15]:
def sortGraphs(graphs):
    newGraphs={}
    for name, g in graphs.items():
        if len(name) == 3:
            newName = name[:-1] + "0" +name[-1] 
            newGraphs[newName] = g
        else:
            newGraphs[name] = g

    newGraphs = OrderedDict(sorted(newGraphs.items()))

    graphs={}
    for name, g in newGraphs.items():
        if name[2] == '0':
            newName = name.replace("0", '')
            graphs[newName] = g
        else:
            graphs[name] = g
    return graphs

In [16]:
def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        for word, weight, weight2 in words[:15]:
            parsedPhrase = ''
            count = 0
            for w in word.split(' '):
                if w != '':
                    parsedWord = stemmer.stem(w.lower())
                    count = count + 1
                    if count > 1:
                        parsedPhrase = parsedPhrase + ' ' + parsedWord
                    else:
                        parsedPhrase = parsedWord

            if(word != words[0][0]):
                file.write("," + parsedPhrase)
            else:
                file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [17]:
def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [18]:
def takeSecond(elem):
    return elem[1]

In [19]:
def keyPhrasesCompilation(keyWords, g, g2, dicionarioCode):
    key2 = {code : value for code, value in keyWords}
    
    keyphrases_all = []
    for k in keyWords[:20]:
        for k2 in keyWords[:20]:
            if k[0] != k2[0]:
                if g2.has_edge(k[0], k2[0]):
                    if g2.has_edge(k2[0], k[0]) and g2[k2[0]][k[0]]['weight'] > g2[k[0]][k2[0]]['weight']:
                        weight = g.out_degree(k2[0], weight='weight') + g.in_degree(k[0], weight='weight')
                        phrase = [k2[0] + ',' + k[0], g2[k2[0]][k[0]]['weight'], g[k2[0]][k[0]]['weight'] / weight]
                    else:
                        weight = g.out_degree(k[0], weight='weight') + g.in_degree(k2[0], weight='weight')
                        phrase = [k[0] + ',' + k2[0], g2[k[0]][k2[0]]['weight'], g[k[0]][k2[0]]['weight'] / weight]
                    if phrase not in keyphrases_all:
                        keyphrases_all.append(phrase)
                        
                        
    keyphrases_weight = [t[2] for t in keyphrases_all if t[1] >= 5]   # only phrases that appeared more than 2 times
    keyphrases_weight_norm = [float(i)/sum(keyphrases_weight) for i in keyphrases_weight]
    keyphrases = [t for t in keyphrases_all if t[1] >= 5]
    
    for kp, n in zip(keyphrases, keyphrases_weight_norm):
        codes = kp[0].split(',')
        #kp[1] = ((key2[codes[0]] + key2[codes[1]] ) / 2 ) / ((1-(n/(peso[0][codes[0]] * peso[0][codes[1]]))) / 1)
        #kp[1] = ((key2[codes[0]] + key2[codes[1]] ) / 2 ) / ((1-(n)) / 1)
        kp[1] = ((key2[codes[0]] + key2[codes[1]]) / 2) + n/2
        #kp[1] = n
        kp[0] = dicionarioCode[codes[0]] + ' ' + dicionarioCode[codes[1]]
        
    
    keywords = [[dicionarioCode[k[0]], k[1], 1] for k in keyWords]
    merged = keyphrases + keywords
    merged.sort(key=takeSecond, reverse=True)
        
    return merged

## Set variables

In [20]:
inputDirectory = "/home/mauro/Downloads/testeSemeval"
outputFile = "/home/mauro/Downloads/testeSemeval/teste.txt"
saveFile = "/home/mauro/Downloads/testeSemeval/teste2.txt"
resultFile = "/home/mauro/Downloads/testeSemeval/resultado.txt"

In [1]:
lang = "EN"
key = "" 

In [22]:
dist = 35
#dist  = float("inf")
#dist = 2

## Create graphs

In [23]:
#splitted_text = []
#dicionario = []
#dicionarioCode = []
#peso = []
#paragraphs_annotations = []
#paragraphs_text = []
#paragraphs_code = []
#graphName = []
graphs = []
graphs2 = []
keyWords = []
keyPhrases = []

## Save KeyWords

In [24]:
saveKeyWords(keyPhrases, graphName, resultFile)

NameError: name 'graphName' is not defined

In [25]:
def nodeRank(g,dicionarioCode):
    #pr = nx.pagerank(g, weight='weight')
    #pr = nx.eigenvector.eigenvector_centrality_numpy(g, weight='weight')
    #pr = nx.closeness_centrality(g, distance='weight')
    #pr = nx.betweenness_centrality(g, weight='weight')
    pr = nx.degree_centrality(g)
    desc = sorted(pr.keys(), key=pr.get,reverse=False)
    pageRank = {}
    for p in pr:
        pageRank[p] = pr[str(p)] 
    pageRank = sorted(pageRank.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
    return pageRank

    #desc = sorted(pr.keys(), key=pr.get,reverse=False)
    #pageRank = {}
    #for p in pr:
    #    pageRank[dicionarioCode[p]] = pr[str(p)] 
    #pageRank = sorted(pageRank.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
    #return pageRank

In [26]:
#Permite respostas iguais
def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        for word, weight, weight2 in words[:15]:
            parsedPhrase = ''
            count = 0
            for w in word.split(' '):
                if w != '':
                    parsedWord = stemmer.stem(w.lower(), 0, len(w)-1)
                    count = count + 1
                    if count > 1:
                        parsedPhrase = parsedPhrase + ' ' + parsedWord
                    else:
                        parsedPhrase = parsedWord

            if(word != words[0][0]):
                file.write("," + parsedPhrase)
            else:
                file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [27]:
#Permite respostas iguais
def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        for word, weight, weight2 in words[:15]:
            parsedPhrase = ''
            for wr in word.split(' '):
                for w,i in zip(wr.split('-'),range(len(wr.split('-')))):
                    if w != '':
                        parsedWord = stemmer.stem(w.lower(), 0, len(w)-1)
                        if i >= 1:
                            parsedPhrase = parsedPhrase + '-' + parsedWord
                        else:
                            parsedPhrase = parsedPhrase + parsedWord
                
                parsedPhrase = parsedPhrase + ' '
            parsedPhrase = parsedPhrase[:len(parsedPhrase)-1]
            parsedPhrase = parsedPhrase.replace('  ',' ')
            
            if(word != words[0][0]):
                file.write("," + parsedPhrase)
            else:
                file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [28]:
# Elimina respostas iguais (piora resultado)

def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        keys = set()
        for word, weight, weight2 in words[:30]:
            parsedPhrase = ''
            for wr in word.split(' '):
                for w,i in zip(wr.split('-'),range(len(wr.split('-')))):
                    if w != '':
                        parsedWord = stemmer.stem(w.lower(), 0, len(w)-1)
                        if i >= 1:
                            parsedPhrase = parsedPhrase + '-' + parsedWord
                        else:
                            parsedPhrase = parsedPhrase + parsedWord
                parsedPhrase = parsedPhrase + ' '
            parsedPhrase = parsedPhrase[:len(parsedPhrase)-1]
            parsedPhrase = parsedPhrase.replace('  ',' ')  
            if parsedPhrase not in keys:
                keys.add(parsedPhrase)

                if(word != words[0][0]):
                    file.write("," + parsedPhrase)
                else:
                    file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [29]:
def defineEdgeWeight(B, G, g):
    weightInB = 0.0
    #weightOutB = 0.0
    
    for u in B:
        for v in G:
            if g.has_edge(u, v):
                weightInB = weightInB + g[u][v]['weight'] 
          #  if g.has_edge(v, u):
          #      weightOutB = weightOutB + g[v][u]['weight']
    return weightInB

In [30]:
def mergeNodes(g1,g2,dicionarioCode):
    
    stemmer=PorterStemmer()
    #regra para união dos vértices (devem ter o mesmo radical)
    same_stem = lambda u, v: (stemmer.stem(dicionarioCode[u].lower(), 0, len(dicionarioCode[u])-1) == stemmer.stem(dicionarioCode[v].lower(), 0, len(dicionarioCode[v])-1))
    
    #regra do peso das uniões (soma de todos os vértices)
    node_weight = lambda G: {'peso' : (sum([g1.nodes[i]['peso'] for i in G]))}
    node_weight2 = lambda G: {'peso' : (sum([g2.nodes[i]['peso'] for i in G]))}
    
    #regra do peso das arestas (soma de todas as arestas)
    edges_weight = lambda B, G: {'weight' : (defineEdgeWeight(B, G, g1))}
    edges_weight2 = lambda B, G: {'weight' : (defineEdgeWeight(B, G, g2))}
    
        
    #merge nodes
    gra1 = nx.quotient_graph(g1,same_stem,node_data=node_weight, edge_data=edges_weight)
    gra2 = nx.quotient_graph(g2,same_stem,node_data=node_weight2, edge_data=edges_weight2)
    
    
    
    #atualiza nome dos vertices
    mappingNodes = {}
    for b in gra1.nodes():
        higherNode = ''
        higherWeight = 0
        for n in b:
            if g1.nodes[n]['peso'] > higherWeight:
                higherWeight = g1.nodes[n]['peso']
                higherNode = n
        mappingNodes[b] = higherNode
        
    grafo1 = nx.relabel_nodes(gra1, mappingNodes)
    
    mappingNodes = {}
    for b in gra2.nodes():
        higherNode = ''
        higherWeight = 0
        for n in b:
            if g2.nodes[n]['peso'] > higherWeight:
                higherWeight = g2.nodes[n]['peso']
                higherNode = n
        mappingNodes[b] = higherNode
        
    grafo2 = nx.relabel_nodes(gra2, mappingNodes)
    
    
    
    return grafo1,grafo2

In [31]:
#Permite respostas iguais
def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        for word, weight, weight2 in words[:15]:
            parsedPhrase = ''
            for wr in word.split(' '):
                for w,i in zip(wr.split('-'),range(len(wr.split('-')))):
                    if w != '':
                        parsedWord = stemmer.stem(w.lower(), 0, len(w)-1)
                        if i >= 1:
                            parsedPhrase = parsedPhrase + '-' + parsedWord
                        else:
                            parsedPhrase = parsedPhrase + parsedWord
                
                parsedPhrase = parsedPhrase + ' '
            parsedPhrase = parsedPhrase[:len(parsedPhrase)-1]
            parsedPhrase = parsedPhrase.replace('  ',' ')
            
            if(word != words[0][0]):
                file.write("," + parsedPhrase)
            else:
                file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [32]:
def createDicts(paragraphs_text, paragraphs_code):
    dicionario={}
    peso={}
    dicionarioCode={}
    for paragraph, codes in zip(paragraphs_text, paragraphs_code):
        for word, code in zip(paragraph, codes):
            if code not in peso:
                peso[code] = 1
            else:
                peso[code] = peso[code] + 1
                
            if word not in dicionario:
                    dicionario[word] = code
            if code not in dicionarioCode:
                    dicionarioCode[code] = word
            #elif len(dicionarioCode[code]) < len(word):
            #    dicionarioCode[code] = word
    return dicionario, dicionarioCode, peso

In [33]:
dicionario = []
dicionarioCode= []
peso = []
for i in range(100):
    d, dC, p = createDicts(paragraphs_text[i],paragraphs_code[i])
    dicionario.append(d)
    dicionarioCode.append(dC)
    peso.append(p)

NameError: name 'paragraphs_text' is not defined

In [34]:
dicionario = {}
for d in dicionarioCode:
    dicionario.update(d)
            


In [35]:
tf = {}
for d in dicionarioCode:
    tf.update(d)
    
for t in tf:
    tf[t] = 0
    for text in paragraphs_code:
        tInText = False
        for p in text:
            if t in p:
                tInText = True
                break
        if tInText:
            tf[t] += 1
            
for t in tf:
    tf[t] = log(len(paragraphs_code) / tf[t])
    

In [39]:
def keyPhrasesCompilation(peso, keyWords, g, g2, dicionarioCode,lenght,totalWords):
    key2 = {code : value for code, value in keyWords}
    
    keyphrases_all = []

        
    for k in keyWords[:lenght]:
        for k2 in keyWords[:lenght]:
            if g2.has_edge(k[0], k2[0]) and g2[k[0]][k2[0]]['weight'] >= int(totalWords / 1000) + 2:
                if g2.has_edge(k2[0], k[0]) == False or g2[k2[0]][k[0]]['weight'] < g2[k[0]][k2[0]]['weight']:
                    weight = g.out_degree(k[0], weight='weight') + g.in_degree(k2[0], weight='weight')
                    phrase = [k[0] + ',' + k2[0], g2[k[0]][k2[0]]['weight'], g[k[0]][k2[0]]['weight'] / weight]
                    if phrase not in keyphrases_all:
                        keyphrases_all.append(phrase)

    keyphrases_all = sorted(keyphrases_all, key=lambda kv: (kv[1], kv[0]), reverse=True) 
    keyphrases_weight = [t[2] for t in keyphrases_all]
    keyphrases_weight_norm = [float(i)/sum(keyphrases_weight) for i in keyphrases_weight]
    keyphrases = [t for t in keyphrases_all]
    
    for kp, n in zip(keyphrases, keyphrases_weight_norm):
        codes = kp[0].split(',')

        kp[1] = ((key2[codes[0]] + key2[codes[1]])) * n

        kp[0] = dicionarioCode[codes[0]] + ' ' + dicionarioCode[codes[1]]
        
    soma = sum([v[1] for v in keyphrases])
    keyphrases = [[k[0], 1 * k[1]/soma, k[2]] for k in keyphrases]   
    keywords = [[dicionarioCode[k[0]], k[1], 1] for k in keyWords]    
    merged = keyphrases[:6] + keywords

    merged.sort(key=takeSecond, reverse=True)
        
    return merged

## Run Experiment

In [40]:
phrases = []
graphs = []
graphs2 = []
#graphs3 = []
dit = 3 #testar com 15
words = []

for i in range(100):
    g, g2, g3 = createGraph(peso[i], paragraphs_code[i], dit)
    #g, g2 = mergeNodes(g,g2,dicionarioCode[i])    
    graphs.append(g)
    graphs2.append(g2)
    
    keywords = nodeRank(graphs[i],dicionarioCode[i])    
    
    
    #Exclui verbos no infinitivo
    key2 = []
    for k in keywords:
        #if graphs2[i].nodes[k[0]]['peso'] > 1:
        palavras = dicionarioCode[i][k[0]]
        tokens = nltk.word_tokenize(palavras)
        verb = False
        for w in nltk.pos_tag(tokens):
            if w[1][0] != 'N' and w[1][0] != 'J' and w[1][0] != 'V':
                verb = True
            #if w[1] == 'VB'
            #    verb = True
        if verb:
            bla = [k[0], 0]
            key2.append(bla)

        else:
            key2.append(k)
             
                
    keywords = []
    keywords = key2
    keywords = sorted(keywords, key=lambda kv: (kv[1], kv[0]), reverse=True)     
    
    lenght = int(.13*len(keywords))
    soma = sum([v[1] for v in keywords[:lenght]])
    keywords = [[k[0], k[1]/soma] for k in keywords[:lenght]]    
    keywords = [[k[0], (k[1]**(1/(len(dicionarioCode[i][k[0]].split(' ')))))] for k in keywords[:lenght]]
    #keywords = [[k[0], (k[1]**(1/(len(re.split(' |-',dicionarioCode[i][k[0]])))))] for k in keywords[:90]]
    
    #keywords = [[k[0], k[1]*tf[k[0]]] for k in keywords[:lenght]] 
    
    keywords = sorted(keywords, key=lambda kv: (kv[1], kv[0]), reverse=True)  
    
    
    firstWords = []
    totalWords = sum([len(i) for i in paragraphs_code[i]])
    
    
    if totalWords > 1000:
        threshold = int(totalWords * .18)
    else:
        threshold = totalWords
    
    for p in paragraphs_code[i]:
        for w in p:
            if len(firstWords) < threshold:
                firstWords.append(w)
            else:
                break
    
    newKeywords = []
    for k in keywords:
        if k[0] in firstWords:
            newKeywords.append(k)
    keywords = newKeywords
    
    nodesToRemove = []
    for n in g:
        inKeywords = False
        for k in keywords:
            if k[0] == n:
                inKeywords = True
        if inKeywords == False:
            nodesToRemove.append(n)
            
    for n in nodesToRemove:
        g.remove_node(n)
        g2.remove_node(n)
        
    
    #soma = sum([v[1] for v in keywords])
    #keywords = [[k[0], k[1]/soma] for k in keywords]    
    #keywords = [[k[0], 3*(k[1]**(1/(len(dicionarioCode[i][k[0]].split(' ')))))] for k in keywords]
    #for k in keywords:
        #if len(dicionarioCode[i][k[0]].split(' ')) <= 1:
        #    k[1] = 0
    
    
    #Test with just 1 keyword
    #for k in keywords:
    #    if len(dicionarioCode[i][k[0]].split(' ')) > 1:
    #        k[1] = 0
    
    
   # keywords = [[k[0], 1*(k[1]*((len(dicionarioCode[i][k[0]].split(' ')))))] for k in keywords[:90]]
    
    #keywords = [[k[0], (k[1] + ((k[1] * (len(dicionarioCode[i][k[0]].split(' ')) - 1))**(1/3)))] for k in keywords]
    #keywords = [[k[0], (k[1] + 1* (35/(0.000000000000001+sum([1 for t in keywords[:35] if len(dicionarioCode[i][t[0]].split(' ')) == len(dicionarioCode[i][k[0]].split(' '))]))) * k[1])] for k in keywords[:35]]
    words.append(keywords)
    keyphrase = keyPhrasesCompilation(peso[i], keywords,graphs[i],graphs2[i],dicionarioCode[i],lenght,totalWords)    
    phrases.append(keyphrase)
    print(i)
saveKeyWords(phrases, graphName, '/home/mauro/Downloads/SemEval2010/test_answer/resultado.txt')
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [295]:
phrases[10]

[['local area network', 0.21417348839739905, 1],
 ['Data Center', 0.18682012208131593, 1],
 ['management  system', 0.17575631692144592, 1],
 ['live migration', 0.09780422798312656, 1],
 ['server Migration', 0.07877944881582494, 0.21414670040441414],
 ['make use', 0.06821706337320382, 1],
 ['server virtualization', 0.06629506915612746, 1],
 ['service providers', 0.06227337402710532, 1],
 ['virtual server', 0.047478834786781166, 0.21642710119101485],
 ['Migration WANs', 0.04558466167391896, 0.16674724380359526],
 ['Migration', 0.04550155118924513, 1],
 ['server', 0.03903826266804554, 1],
 ['applications', 0.029472595656670136, 1],
 ['physical server', 0.024959349731658188, 0.1282429102980701],
 ['Internet-based', 0.024301964839710467, 1],
 ['services', 0.02042399172699071, 1],
 ['storage', 0.019906928645294746, 1],
 ['Approach', 0.019906928645294746, 1],
 ['data', 0.01809720785935886, 1],
 ['WANs', 0.017321613236814908, 1],
 ['replication', 0.015770423991727005, 1],
 ['technologies', 0.0

In [167]:
saveKeyWords(phrases, graphName, '/home/mauro/Downloads/SemEval2010/test_answer/resultado.txt')

In [53]:
phrases[1]

IndexError: list index out of range

In [54]:
phrases[0]

IndexError: list index out of range

In [55]:
indice = 2
for p in phrases[indice]:
    tokens = nltk.word_tokenize(p[0])
    texto = ""
    for w in nltk.pos_tag(tokens):
        print(w)
    
    #for word in tokens:
    #    texto = texto + word + ' ' + 
        #palavras = dicionarioCode[i][k[0]]
        #tokens = nltk.word_tokenize(palavras)
        #for t in tokens:

IndexError: list index out of range

In [56]:
for w in words[7]:
    print(dicionarioCode[7][w[0]] + ' ' + str(w[1]))

IndexError: list index out of range

In [57]:
def saveKeyWords(keyWords, graphName, resultFile):
    file = open(resultFile,"w+") 
    stemmer=PorterStemmer()
    for words, name in zip(keyWords, graphName):
        file.write(name + " : ")
        #for word in [it[0] for it in nltk.pos_tag([item[0] for item in words]) if it[1] != 'VBN'][:15]:
        for word, weight, weight2 in words[:15]:
            parsedPhrase = ''
            count = 0
            for w in word.split(' '):
                if w != '':
                    parsedWord = stemmer.stem(w.lower())
                    count = count + 1
                    if count > 1:
                        parsedPhrase = parsedPhrase + ' ' + parsedWord
                    else:
                        parsedPhrase = parsedWord

            if(word != words[0][0]):
                file.write("," + parsedPhrase)
            else:
                file.write(parsedPhrase)
        file.write("\n")
    file.close()
    return 

In [58]:
count = 0
for k in words[7]:
    count = count + 1
    print(str(count) + '\t' + dicionarioCode[3][k[0]] + '\t' + str(k[1]))

IndexError: list index out of range

In [59]:
stemmer=PorterStemmer()


In [60]:
peso[0]['bn:00819565n']

IndexError: list index out of range

In [61]:
graphs[0].node('bn:03664545n', weight=True)

IndexError: list index out of range

In [62]:
dicionarioCode[0]['bn:03664545n']

IndexError: list index out of range

In [63]:
dicionario[0]['UDDI']

KeyError: 0

In [64]:
graphs2[0]['bn:00041777n']['bn:00395755n']

IndexError: list index out of range

In [65]:
graphs2[0].has_edge('bn:00102206a', 'bn:00036197n')

IndexError: list index out of range

In [66]:
for u, v, weight in graphs[0].edges(data='weight'):
    if u == 'bn:00070651n' and weight > 5 and u != v:
        print(dicionarioCode[0][v] + ' ' + str(weight))

IndexError: list index out of range

# Babelfy text

In [20]:
key = "" 

In [21]:

graphName = []
splitted_text = []
dicionario = []
dicionarioCode = []
peso = []
paragraphs_annotations = []
paragraphs_text = []
paragraphs_code = []
graphs = []
graphs2 = []
keyWords = []
keyPhrases = []
inputDirectory = "/home/mauro/Downloads/SemEval2010/test/"


In [ ]:
def startDownload():
    count = 0
    atual = 81
    for filename in sorted(glob.glob(os.path.join(inputDirectory, '*.txt.final')), key=numericalSort):
        if count >= atual:
            graphName.append(filename[39] + '-' + str(re.findall(r'\d+', filename[39:])[0]))

            text = openFile(filename) 
            st = parseText(text)
            splitted_text.append(st)

            pa, pt, pc  = babelfy(lang, key, st)
            paragraphs_annotations.append(pa)
            paragraphs_text.append(pt)
            paragraphs_code.append(pc)
            #saveText(saveFile, paragraphs_code)

            d, dc, p = createDicts(pt, pc)
            dicionario.append(d)
            dicionarioCode.append(dc)
            peso.append(p)
        count = count + 1
        print('Artigo: ' + str(count))


## Run download in background

In [ ]:
import threading

In [ ]:
thr = threading.Thread(target=startDownload, args=())

In [ ]:
thr.daemon = True

In [ ]:
thr.start()

In [ ]:
thr.is_alive()

In [ ]:
len(splitted_text)

In [ ]:
len(paragraphs_annotations)

In [ ]:
#splitted_text.pop(69)
#graphName.pop(69)

# Save babelfyed text

In [32]:
saveFile = '/home/mauro/Documentos/Mestrado/babelfyLists/'

In [127]:
''''with open(saveFile + "graphName.pickle", "wb") as fp:
    pickle.dump(graphName, fp)
with open(saveFile + "splitted_text.pickle", "wb") as fp:
    pickle.dump(splitted_text, fp)
with open(saveFile + "dicionario.pickle", "wb") as fp:
    pickle.dump(dicionario, fp)
with open(saveFile + "dicionarioCode.pickle", "wb") as fp:
    pickle.dump(dicionarioCode, fp)
with open(saveFile + "peso.pickle", "wb") as fp:
    pickle.dump(peso, fp)
with open(saveFile + "paragraphs_annotations.pickle", "wb") as fp:
    pickle.dump(paragraphs_annotations, fp)
with open(saveFile + "paragraphs_text.pickle", "wb") as fp:
    pickle.dump(paragraphs_text, fp)
with open(saveFile + "paragraphs_code.pickle", "wb") as fp:
    pickle.dump(paragraphs_code, fp)'''
with open(saveFile + "graphsDistINF.pickle", "wb") as fp:
    pickle.dump(graphs, fp)
with open(saveFile + "graphs2DistINF.pickle", "wb") as fp:
    pickle.dump(graphs2, fp)

## Open babelfyed text

In [38]:
graphName = []
splitted_text = []
dicionario = []
dicionarioCode = []
peso = []
paragraphs_annotations = []
paragraphs_text = []
paragraphs_code = []
graphs = []
graphs2 = []
keyWords = []
keyPhrases = []
inputDirectory = "/home/mauro/Downloads/SemEval2010/test/"
saveFile = '/home/mauro/Documentos/Mestrado/babelfyLists/'

with open (saveFile + "graphName.pickle", 'rb') as fp:
    graphName = pickle.load(fp)
with open (saveFile + "splitted_text.pickle", 'rb') as fp:
    splitted_text = pickle.load(fp)
with open (saveFile + "dicionario.pickle", 'rb') as fp:
    dicionario = pickle.load(fp)
with open (saveFile + "dicionarioCode.pickle", 'rb') as fp:
    dicionarioCode = pickle.load(fp)
with open (saveFile + "peso.pickle", 'rb') as fp:
    peso = pickle.load(fp)
with open (saveFile + "paragraphs_annotations.pickle", 'rb') as fp:
    paragraphs_annotations = pickle.load(fp)
with open (saveFile + "paragraphs_text.pickle", 'rb') as fp:
    paragraphs_text = pickle.load(fp)
with open (saveFile + "paragraphs_code.pickle", 'rb') as fp:
    paragraphs_code = pickle.load(fp)
with open (saveFile + "graphsDist4.pickle", 'rb') as fp:
    graphs = pickle.load(fp)
with open (saveFile + "graphs2Dist4.pickle", 'rb') as fp:
    graphs2 = pickle.load(fp)